In [1]:
pip install streamlit plotly pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install streamlit pyngrok

In [3]:
code = '''
import streamlit as st
import pandas as pd
import plotly.express as px

@st.cache_data
def load_data():
    df = pd.read_csv("superstore.csv", encoding="ISO-8859-1")
    df.columns = df.columns.str.replace('.', '_').str.strip()
    df = df.drop(columns=['è®°å½æ°'], errors='ignore')
    df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')
    df['Ship_Date'] = pd.to_datetime(df['Ship_Date'], errors='coerce')
    return df

df = load_data()

st.sidebar.header("🔍 Filter Options")
regions = st.sidebar.multiselect("Select Region", df['Region'].unique(), default=df['Region'].unique())
categories = st.sidebar.multiselect("Select Category", df['Category'].unique(), default=df['Category'].unique())
sub_categories = st.sidebar.multiselect("Select Sub-Category", df['Sub_Category'].unique(), default=df['Sub_Category'].unique())

filtered_df = df[
    (df['Region'].isin(regions)) &
    (df['Category'].isin(categories)) &
    (df['Sub_Category'].isin(sub_categories))
]

st.title("📊 Global Superstore Business Dashboard")

total_sales = filtered_df['Sales'].sum()
total_profit = filtered_df['Profit'].sum()
top_customers = (
    filtered_df.groupby('Customer_Name')['Sales']
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

col1, col2 = st.columns(2)
col1.metric("💰 Total Sales", f"${total_sales:,.2f}")
col2.metric("📈 Total Profit", f"${total_profit:,.2f}")

st.subheader("🏆 Top 5 Customers by Sales")
fig1 = px.bar(
    top_customers.sort_values(),
    x=top_customers.sort_values().values,
    y=top_customers.sort_values().index,
    orientation='h',
    labels={'x': 'Sales', 'y': 'Customer'},
    color=top_customers.sort_values().values,
    height=400
)
st.plotly_chart(fig1, use_container_width=True)

st.subheader("💹 Sales vs Profit by Sub-Category")
fig2 = px.scatter(
    filtered_df, x='Sales', y='Profit', color='Sub_Category',
    size='Sales', hover_data=['Product_Name', 'Customer_Name'],
    height=500
)
st.plotly_chart(fig2, use_container_width=True)

st.subheader("📦 Sales Distribution by Category")
category_sales = filtered_df.groupby('Category')['Sales'].sum().reset_index()
fig3 = px.pie(
    category_sales,
    values='Sales',
    names='Category',
    title='Sales by Category',
    hole=0.4
)
st.plotly_chart(fig3, use_container_width=True)
'''

with open("dashboard.py", "w") as f:
    f.write(code)

In [12]:
!ngrok config add-authtoken 30FXmvJLoCBj3KHDDjvnMJo3RVj_yXWcdULXwt7fwfMFKnDf

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from pyngrok import ngrok
import os

# Kill any previous Streamlit processes (optional)
!pkill streamlit

# Start a tunnel to port 8501
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)

# Run Streamlit app
!streamlit run dashboard.py &>/dev/null &

Streamlit URL: NgrokTunnel: "https://0993d954cd5f.ngrok-free.app" -> "http://localhost:8501"


StreamKit is working Perfect